In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Adataset_new - final.csv")

In [ ]:
df['category'].value_counts()

In [ ]:
categories = [
    "वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी",
    "पार्किङ्ग तथा मेट्रो पुलिस",
    "सोधपुछ, सुझाव, प्रशंसा सम्बन्धी",
    "कर्मचारी सम्वन्धी",
    "खानेपानी",
    "अर्थ सबन्धी",
    "शान्ति सुरक्षा",
    "स्वास्थ्यसँग सम्बन्धी",
    "सूचना तथा संचार",
    "फोहोरमैला व्यवस्थापन",
    "निर्माण कार्य सम्बन्धी",
    "प्राकृतिक श्रोत/साधन सम्बन्धी",
    "लागु पदार्थ सम्बन्धी",
    "अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी"
]


In [ ]:

df_final = pd.DataFrame()  # Initialize an empty DataFrame to hold the final result

for category in categories:
    # Select rows matching the current category
    temp_df = df[df['category'] == category]

    # If the category requires sampling, perform sampling; otherwise, proceed as is
    if category != 'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी':  # Adjust this condition as needed
        temp_df = temp_df.sample(2863, replace=True).reset_index(drop=True)

    df_final = pd.concat([df_final, temp_df], axis=0)

# Optionally, reset the index of the final DataFrame
df_final = df_final.reset_index(drop=True)


In [ ]:
df_final.value_counts()

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')  # For English stopwords
print(stop_words)  # This will print the list of English stopwords


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Data Preprocessing
nepali_stopwords = set(stopwords.words('nepali'))
english_stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[#\\/।(),०-९<<?!,—–’‘:\u200d]', '', text)
    text = text.strip('"')
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in nepali_stopwords and word.lower() not in english_stopwords]
    processed_text = ' '.join(filtered_words)
    return processed_text

# df_final['complain'] = df_final['complain'].apply(preprocess_text)

In [ ]:
# Encoding Labels
le = LabelEncoder()
df_final['label_encoded'] = le.fit_transform(df_final['category'])


In [ ]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df_final, test_size=0.2, stratify=df_final['label_encoded'], random_state=42)


In [ ]:
# Tokenize the text data
max_words = 10000
max_length = 100
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['complain'])

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_data['complain'])
test_sequences = tokenizer.texts_to_sequences(test_data['complain'])

In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
# Load your pre-trained Nepali Word2Vec model
word2vec_model_path = "/content/drive/MyDrive/nepaliW2V_5Million.model"
word2vec_model = Word2Vec.load(word2vec_model_path)

In [ ]:
# Create an embedding matrix
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words and word in word2vec_model.wv:

        embedding_matrix[i] = word2vec_model.wv[word]

In [ ]:
# Build the Bi-LSTM model with Word2Vec embeddings
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(Bidirectional(LSTM(units=50)))
model.add(Dense(units=len(np.unique(df_final['label_encoded'])), activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 20
history=model.fit(train_padded, train_data['label_encoded'], epochs=epochs, validation_data=(test_padded, test_data['label_encoded']))


In [ ]:
import matplotlib.pyplot as plt

# Data
epochs = range(1, 21)
accuracy = [0.7126, 0.8289, 0.8817, 0.9101, 0.9331, 0.9414, 0.9560, 0.9592, 0.9608, 0.9721, 0.9692, 0.9736, 0.9756, 0.9736, 0.9796, 0.9785, 0.9767, 0.9766, 0.9819, 0.9820]
f1_score = [0.7909, 0.8445, 0.8770, 0.9028, 0.9030, 0.9176, 0.9168, 0.9239, 0.9314, 0.9292, 0.9246, 0.9337, 0.9310, 0.9299, 0.9374, 0.9361, 0.9176, 0.9390, 0.9301, 0.9367]

# Plotting
plt.figure(figsize=(10, 6))

plt.plot(epochs, accuracy, label='Accuracy', color='tab:blue', marker='o')
plt.plot(epochs, f1_score, label='F1-score', color='tab:orange', linestyle='--', marker='o')

plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.title('Accuracy and F1-score over Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Data
epochs = range(1, 21)
training_loss = [0.9150, 0.5152, 0.3542, 0.2689, 0.1997, 0.1737, 0.1318, 0.1159, 0.1162, 0.0836, 0.0890, 0.0764, 0.0688, 0.0777, 0.0573, 0.0582, 0.0666, 0.0636, 0.0488, 0.0510]
validation_loss = [0.6406, 0.4726, 0.3876, 0.3118, 0.3157, 0.2692, 0.2726, 0.2639, 0.2527, 0.2728, 0.2890, 0.2729, 0.2830, 0.2836, 0.2871, 0.2881, 0.3491, 0.2676, 0.3001, 0.2996]

# Plotting
plt.figure(figsize=(10, 6))

plt.plot(epochs, training_loss, label='Training Loss', color='tab:blue', marker='o')
plt.plot(epochs, validation_loss, label='Validation Loss', color='tab:orange', linestyle='--', marker='o')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation accuracies
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Evaluate the model
y_pred_probs = model.predict(test_padded)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Decode the encoded labels back to original classes
y_true = le.inverse_transform(test_data['label_encoded'])
y_pred_decoded = le.inverse_transform(y_pred)

# Print the classification report
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(y_true, y_pred_decoded))


In [ ]:
model.save("/content/drive/MyDrive/model.h5")

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/content/drive/MyDrive/model.h5')



In [ ]:
# New Nepali text
new_text = "यस महानगरपालिकाको सुनाकोठी नखिपोट नख्खुडोल गोकुल आवास कान्तिपुर कोलोनीका वासिन्दाले दिनरात यो धुंवाको सास कहिलेसम्म फेर्नुपर्ने हो हजुर?"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = loaded_model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

In [ ]:
# New Nepali text
new_text = "सुनसरी जिल्ला इटहरी उप महानगर भित्र अवस्थित धरान पुग्ने यो बाटो । कुन ठेकेदारले यो काम गर्दा कति पैसा कुम्ल्याउन पाइने हो र आफ्नो निर्वाचन ताका खर्च भएको करोड उठाउन पाइन्छ भनेर त हैन ?"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

In [ ]:
# New Nepali text
new_text = "काठमाडौं मा मेलम्ची को पानी बितरन्ण भयो भन्छ । हामी मा अहीपुग्ने कहिले हो मेरो मा पानी न आहेक्क आजको दिन टक्काइ ४० दिन भयो । खानी पानी कार्यल मा भनो भने पाले ले आजै दिन्छ भन्छ हामी लाई फर्काइ दिन्छ अब भन्नुस् कति दिन यो महङ्गो मा पानी किनेर जिबन याबन गर्ने । हजुर हरु लाई के लाग्छ के हामी पानी बिना को जिबन जिउन सकिन्छ त ।कृपया हाम्रो समस्य बुझीदिन्छ कि भनेर हजुर सम्म यो गुनासो पठाउन बात्य भएको छु ।"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

In [ ]:
# New Nepali text
new_text = "नगरिक ऐप नागरिकता संशोधन"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

In [ ]:
!pip install nepali-stemmer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
new_text = "काठमाडौं मा मेलम्ची को पानी बितरन्ण भयो भन्छ । हामी मा अहीपुग्ने कहिले हो मेरो मा पानी न आहेक्क आजको दिन टक्काइ ४० दिन भयो । खानी पानी कार्यल मा भनो भने पाले ले आजै दिन्छ भन्छ हामी लाई फर्काइ दिन्छ अब भन्नुस् कति दिन यो महङ्गो मा पानी किनेर जिबन याबन गर्ने । हजुर हरु लाई के लाग्छ के हामी पानी बिना को जिबन जिउन सकिन्छ त ।कृपया हाम्रो समस्य बुझीदिन्छ कि भनेर हजुर सम्म यो गुनासो पठाउन बात्य भएको छु ।"


In [ ]:
from nepali_stemmer.stemmer import NepStemmer
nepstem = NepStemmer()
nepstem.stem(new_text)


'काठमाडौं मा मेलम्ची को पानी बितरन्ण भयो भन्छ । हामी मा अहीपुग्ने कहिले हो मेरो मा पानी न आहेक्क आज को दिन टक्काइ ४० दिन भयो । खानी पानी कार्यल मा भनो भने पाले ले आजै दिन्छ भन्छ हामी लाई फर्काइ दिन्छ अब भन्नुस् कति दिन यो महङ्गो मा पानी किनेर जिबन याबन गर्ने । हजुर हरु लाई के लाग्छ के हामी पानी बिना को जिबन जिउन सकिन्छ त ।कृपया हाम्रो समस्य बुझीदिन्छ कि भनेर हजुर सम्म यो गुनासो पठाउन बात्य भए को छु ।'

In [ ]:
import re
preprocess_text(new_text)

'काठमाडौं मेलम्ची पानी बितरन्ण भयो भन्छ हामी अहीपुग्ने कहिले पानी आहेक्क दिन टक्काइ दिन भयो खानी पानी कार्यल भनो पाले आजै दिन्छ भन्छ हामी फर्काइ दिन्छ भन्नुस् कति दिन महङ्गो पानी किनेर जिबन याबन हजुर हरु लाग्छ हामी पानी बिना जिबन जिउन सकिन्छ हाम्रो समस्य बुझीदिन्छ भनेर हजुर गुनासो पठाउन बात्य'

In [ ]:
import snowballstemmer
nepali_stemmer = snowballstemmer.NepaliStemmer()
y=new_text.split(" ")
print(y)
nepali_stemmer.stemWords(y)



['काठमाडौं', 'मा', 'मेलम्ची', 'को', 'पानी', 'बितरन्ण', 'भयो', 'भन्छ', '।', 'हामी', 'मा', 'अहीपुग्ने', 'कहिले', 'हो', 'मेरो', 'मा', 'पानी', 'न', 'आहेक्क', 'आजको', 'दिन', 'टक्काइ', '४०', 'दिन', 'भयो', '।', 'खानी', 'पानी', 'कार्यल', 'मा', 'भनो', 'भने', 'पाले', 'ले', 'आजै', 'दिन्छ', 'भन्छ', 'हामी', 'लाई', 'फर्काइ', 'दिन्छ', 'अब', 'भन्नुस्', 'कति', 'दिन', 'यो', 'महङ्गो', 'मा', 'पानी', 'किनेर', 'जिबन', 'याबन', 'गर्ने', '।', 'हजुर', 'हरु', 'लाई', 'के', 'लाग्छ', 'के', 'हामी', 'पानी', 'बिना', 'को', 'जिबन', 'जिउन', 'सकिन्छ', 'त', '।कृपया', 'हाम्रो', 'समस्य', 'बुझीदिन्छ', 'कि', 'भनेर', 'हजुर', 'सम्म', 'यो', 'गुनासो', 'पठाउन', 'बात्य', 'भएको', 'छु', '।']


['काठमाडौं',
 '',
 'मेलम्ची',
 '',
 'पानी',
 'बितरन्ण',
 '',
 'भन्',
 '।',
 'हामी',
 '',
 'अहीपुग्',
 'कहि',
 'हो',
 'मेरो',
 '',
 'पानी',
 'न',
 'आहेक्क',
 'आज',
 'दिन',
 'टक्काइ',
 '४०',
 'दिन',
 '',
 '।',
 'खानी',
 'पानी',
 'कार्यल',
 '',
 'भनो',
 'भ',
 'पा',
 '',
 'आजै',
 'द',
 'भन्',
 'हामी',
 '',
 'फर्काइ',
 'द',
 'अब',
 'भन्नुस्',
 'कति',
 'दिन',
 '',
 'महङ्गो',
 '',
 'पानी',
 'किनेर',
 'जिबन',
 'याबन',
 'गर्',
 '।',
 'हजुर',
 '',
 '',
 'के',
 'लाग्',
 'के',
 'हामी',
 'पानी',
 'बिना',
 '',
 'जिबन',
 'जिउन',
 'सक',
 'त',
 '।कृपया',
 'हाम्रो',
 'समस्य',
 'बुझीद',
 '',
 'भनेर',
 'हजुर',
 'सम्म',
 '',
 'गुनासो',
 'पठाउन',
 'बात्य',
 'भ',
 '',
 '।']

In [ ]:
import snowballstemmer
nepali_stemmer = snowballstemmer.NepaliStemmer()
nepali_stemmer.stemWords(["नेपालीको", "आफ्नै", "स्तेम्मेर"])
# ['नेपाली', 'आफ्नै', 'स्तेम्मेर']

['नेपाली', 'आफ्नै', 'स्तेम्मेर']